In [ ]:
import pandas as pd
import numpy as np
import jieba
jieba.set_dictionary('/Users/henryyang/utils/dict.big.txt')

In [ ]:
# Preprocessing (seg)
# Comebine test data & review 
test_data = pd.read_csv('data/test.csv')
review = {}
with open('data/test_review.txt') as f:
    lines = [line.strip() for line in f.readlines()]
    for i in range(0, len(lines), 2):
        review[int(lines[i])] = ' '.join([tok for tok in jieba.cut(lines[i+1])])

test_data['Review'] = [review[review_id] for review_id in test_data.Review_id]
test_data.to_csv('data/test_seg.csv', index=None)

# Seg polarity review
review, polarity = [], []
with open('data/polarity_review.txt') as f:
    for line in [line.strip() for line in f.readlines()]:
        l = line.split('\t')
        polarity.append(int(l[0]))
        review.append(' '.join([tok for tok in jieba.cut(l[1])]))
pd.DataFrame({'Polarity': polarity, 'Review': review}).to_csv('data/polarity_review_seg.csv', index=None)

# Seg aspect review
aspect_review = []
with open('data/aspect_review.txt') as f:
    lines = [line.strip() for line in f.readlines()]
    for i in range(0, len(lines), 4):
        aspect_review.append({
                'Id': int(lines[i]),
                'Review': ' '.join([tok for tok in jieba.cut(lines[i+1])]),
                'Pos': lines[i+2],
                'Neg': lines[i+3]
            })
pd.DataFrame(aspect_review).to_csv('data/aspect_review_seg.csv', index=None)